In [1]:
from kronecker import KroneckerSolver
import kernels as kern
from likelihoods import PoissonLike, GaussianLike
import data_utils as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from IPython.display import display
init_notebook_mode(connected=True)
import numpy as np
from kernels import RBF
from grid_utils import fill_grid
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
import pandas as pd

In [2]:
counts = pd.read_csv('data/zip_counts.csv')
counts = counts[(counts['LAT'] > 20) & (counts['LAT'] < 50) & (counts['LNG'] > -130) & (counts['LNG'] < -70) &(counts['population'] > 0)]
step = .5
to_bin = lambda x: np.floor(x / step) * step
counts["latbin"] = counts.LAT.map(to_bin)
counts["lonbin"] = counts.LNG.map(to_bin)
grouped = pd.DataFrame(counts.groupby(['latbin', 'lonbin', 'committee_name'], as_index = False)['count', 'population'].sum())
clinton = grouped[grouped['committee_name'] == 'DONALD J. TRUMP FOR PRESIDENT, INC.']
X_clinton = np.array(clinton[['latbin','lonbin']])
y_clinton = np.array(clinton['count'], dtype = np.float32)
iplot([go.Scatter3d(x = X_clinton[:,1], y = X_clinton[:,0], z=y_clinton, mode = 'markers', marker=dict(size = 2,))])

In [8]:
X_full, y_full, obs_idx, imag_idx = fill_grid(X_clinton, y_clinton)
k_diag = np.ones(len(y_full))
k_diag[imag_idx] = 1e5
mu = np.ones(len(y_full))*np.mean(np.log(y_full[obs_idx]))
mu[imag_idx] = 0.
y_full_tf = tfe.Variable(y_full, dtype = tf.float32)
mask = tf.less(k_diag, 10)
idx = np.zeros(len(y_full))
idx[obs_idx] = 1.0

In [12]:
ks = KroneckerSolver(tf.constant(mu, tf.float32), RBF(variance=2., length_scale=3.),
                     PoissonLike(), X_full, y_full_tf, 0.5, k_diag = tf.constant(k_diag, tf.float32), mask = mask, verbose = True)
ks.run(10)
iplot([go.Scatter3d(x = X_full[obs_idx,1], y = X_full[obs_idx,0], z=ks.f.numpy()[obs_idx], mode = 'markers', marker=dict(size = 2))])

Iteration:  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
 psi:  tf.Tensor(-2.46527e+06, shape=(), dtype=float32)
step 0.00390625

Iteration:  tf.Tensor(1, shape=(), dtype=int32)
 psi:  tf.Tensor(-3.18383e+06, shape=(), dtype=float32)
step 0.0

Iteration:  tf.Tensor(2, shape=(), dtype=int32)
 psi:  tf.Tensor(-3.18383e+06, shape=(), dtype=float32)
step 0.0

